## Imports
Execute the cell below to import all packages needed.

In [1]:
# System functionality
import sys
sys.path.append('..')

import glob
import os
import numpy as np 
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import seaborn as sns
import graphviz
import math as math
import json
import numpy
import cv2
import scipy.stats as stats
import matplotlib.image as mpimg
import sklearn.feature_selection as feature_selection
from skimage import data, io, filters
from numpy import linspace
from numpy.core.umath import pi
from matplotlib.ticker import FuncFormatter
from matplotlib.font_manager import FontProperties
from emoji_extractor.extract import Extractor
from emojipedia import Emojipedia
from skimage import io
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import chi2_contingency
from scipy.interpolate import spline

## Load csv into pandas dataframe

In [2]:
path__tweets_csv = '/home/stelios/Desktop/Honours Project/Samples/tweet_data/tweets_lon_all.csv'
path_users_csv = '/home/stelios/Desktop/Honours Project/Samples/user_data/lon.csv'

lon_join_df = pd.read_csv(path__tweets_csv,index_col=None, header=0, low_memory = False).applymap(str).join(pd.read_csv(path_users_csv,index_col=None, header=0, low_memory = False).applymap(str).set_index('user_id'), on='user_id')

## Extract, Count & Sort Emojis for all

In [3]:
extract = Extractor()

count = extract.count_all_emoji(lon_join_df['text'])

# Create panda dataframe to store emojis with counts
emojis_count_df = pd.DataFrame(columns = ['Emoji','Count'])
for emoji in count:
    emojis_count_df = emojis_count_df.append({'Emoji' : emoji , 'Count' : count[emoji]} , ignore_index=True)

# Sort dataframe by count
emojis_count_df = emojis_count_df.sort_values(by=['Count'])
emojis_count_df.tail(20)

,Emoji,Count
52,💙,5224
33,😅,6049
25,😁,7146
0,😉,7402
34,🎉,7477
217,💕,7961
57,🙄,8248
53,👀,8372
28,🤔,8532
88,👍,9408


## TABLE

In [4]:
# Load users
lon_users_df = pd.read_csv(path_users_csv,index_col=None, header=0, low_memory = False).applymap(str)

# Calculate lengths
length_emojis = len(emojis_count_df['Emoji'].values)
print(length_emojis)
length_users = len(lon_users_df['user_id'])

# Create zeros matrix
zeros_matrix= numpy.zeros(shape=(length_users,length_emojis))

# Fill pandas dataframe with zeros
emojis_count_create_df = pd.DataFrame(zeros_matrix,columns = emojis_count_df['Emoji'].values)
emojis_count_create_df

# Initialise extractor
extract = Extractor()

# Add the frequencies of emojis each emoji by User
for enum,user in enumerate (lon_users_df['user_id']):
    user_posts= "".join(lon_join_df.loc[lon_join_df['user_id'] == user, 'text'])
    text_count_emoji = extract.count_emoji(user_posts, check_first=True)
    if(len(text_count_emoji)==0):
        pass
    else:
        for emoji in text_count_emoji:
            emojis_count_create_df.at[enum, emoji] = text_count_emoji[emoji]

# Describe
emojis_count_create_df.info(memory_usage='deep')

2213
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Columns: 2241 entries, 🧒🏽 to 🇲🇱
dtypes: float64(2241)
memory usage: 171.0 MB


In [5]:
# Fill n/a with 0.0
emojis_count_create_df = emojis_count_create_df.fillna(0.0)

# Store table
emojis_count_create_df.to_csv("/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/london_per_user_emoji_counts.csv",encoding='utf-8', index=False)

In [6]:
# Testing
emojis_count_create_df['😂']

0       249.0
1         2.0
2         0.0
3         3.0
4         1.0
5         2.0
6         9.0
7       105.0
8         0.0
9         0.0
10        0.0
11       86.0
12        0.0
13        9.0
14        0.0
15        4.0
16        0.0
17        0.0
18        0.0
19        0.0
20        0.0
21        0.0
22        0.0
23        4.0
24        0.0
25        3.0
26        0.0
27       24.0
28      107.0
29        0.0
        ...  
9970      0.0
9971      0.0
9972      0.0
9973      0.0
9974      0.0
9975      0.0
9976     49.0
9977    186.0
9978      1.0
9979      2.0
9980      1.0
9981      0.0
9982      2.0
9983      0.0
9984    293.0
9985      0.0
9986      0.0
9987     10.0
9988      6.0
9989      0.0
9990      0.0
9991      0.0
9992      0.0
9993      0.0
9994      0.0
9995      8.0
9996      0.0
9997     35.0
9998      0.0
9999      0.0
Name: 😂, Length: 10000, dtype: float64